In [1]:
import numpy as np
from collections import Counter

In [2]:
class Node:
    # Initialisation d'un nœud dans un arbre de décision.
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        """
        :param feature: Index de la caractéristique (colonne) utilisée pour diviser les données (None pour les feuilles).
        :param threshold: Seuil utilisé pour diviser les données basées sur la caractéristique (None pour les feuilles).
        :param left: Nœud enfant gauche, représentant les données qui satisfont la condition (valeurs <= seuil).
        :param right: Nœud enfant droit, représentant les données qui ne satisfont pas la condition (valeurs > seuil).
        :param value: Valeur attribuée au nœud si c'est un nœud feuille (ex. une classe pour un problème de classification).
        """
        self.feature = feature  # Caractéristique à tester à ce nœud (None pour un nœud feuille).
        self.threshold = threshold  # Seuil de division basé sur la caractéristique (None pour un nœud feuille).
        self.left = left  # Sous-arbre gauche (None si pas encore défini).
        self.right = right  # Sous-arbre droit (None si pas encore défini).
        self.value = value  # Valeur du nœud feuille (None si ce n'est pas une feuille).
        
    def is_leaf_node(self):
        """
        Vérifie si ce nœud est une feuille.
        Un nœud est considéré comme une feuille s'il contient une valeur spécifique (value n'est pas None).
        """
        return self.value is not None


In [3]:
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        """
        :param min_samples_split: Nombre minimum d'échantillons requis pour diviser un nœud.
        :param max_depth: Profondeur maximale de l'arbre.
        :param n_features: Nombre de caractéristiques à considérer lors de la recherche de la meilleure division.
        """
        self.min_samples_split = min_samples_split  # Critère d'arrêt basé sur le nombre d'échantillons.
        self.max_depth = max_depth  # Limite la profondeur de l'arbre.
        self.n_features = n_features  # Nombre de caractéristiques à utiliser pour la division.
        self.root = None  # Racine de l'arbre, qui sera définie après l'entraînement.

    def fit(self, X, y):
        """
        Entraîne l'arbre de décision en construisant un arbre à partir des données d'entrée X et y.
        """
        # Détermine le nombre de caractéristiques à utiliser (soit toutes, soit limitées par n_features).
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        # Construit l'arbre de manière récursive.
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        """
        Construit récursivement l'arbre de décision en divisant les données.
        :param X: Données d'entrée.
        :param y: Labels correspondants.
        :param depth: Profondeur actuelle du nœud.
        """
        n_samples, n_feats = X.shape  # Nombre d'échantillons et de caractéristiques.
        n_labels = len(np.unique(y))  # Nombre de classes uniques dans les labels.

        # Critères d'arrêt : profondeur maximale atteinte, un seul label, ou trop peu d'échantillons.
        if depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split:
            leaf_value = self._most_common_label(y)  # Crée un nœud feuille avec la classe majoritaire.
            return Node(value=leaf_value)

        # Sélectionne aléatoirement un sous-ensemble des caractéristiques.
        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # Trouve la meilleure division.
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # Divise les données en deux groupes : gauche et droite.
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        # Construit les sous-arbres de manière récursive.
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feature, best_thresh, left, right)

    def _best_split(self, X, y, feat_idxs):
        """
        Trouve la meilleure division en maximisant le gain d'information.
        :param X: Données d'entrée.
        :param y: Labels correspondants.
        :param feat_idxs: Indices des caractéristiques à considérer.
        """
        best_gain = -1  # Initialisation du meilleur gain d'information.
        split_idx, split_threshold = None, None  # Meilleure caractéristique et seuil.

        # Teste chaque caractéristique sélectionnée.
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]  # Colonne correspondant à une caractéristique.
            thresholds = np.unique(X_column)  # Valeurs uniques pour le seuil.

            for thr in thresholds:
                # Calcule le gain d'information pour chaque seuil.
                gain = self._information_gain(y, X_column, thr)

                # Met à jour si un meilleur gain est trouvé.
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold

    def _information_gain(self, y, X_column, threshold):
        """
        Calcule le gain d'information pour une division donnée.
        :param y: Labels.
        :param X_column: Colonne des données pour une caractéristique.
        :param threshold: Seuil pour la division.
        """
        # Entropie parent (avant division).
        parent_entropy = self._entropy(y)

        # Divise les données en deux groupes : gauche et droite.
        left_idxs, right_idxs = self._split(X_column, threshold)

        # Si l'une des divisions est vide, le gain est nul.
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # Calcule l'entropie pondérée des enfants.
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # Calcule le gain d'information.
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        """
        Divise les données en deux groupes basés sur le seuil.
        :param X_column: Colonne des données.
        :param split_thresh: Seuil de division.
        """
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()  # Indices pour le groupe gauche.
        right_idxs = np.argwhere(X_column > split_thresh).flatten()  # Indices pour le groupe droit.
        return left_idxs, right_idxs

    def _entropy(self, y):
        """
        Calcule l'entropie d'un ensemble de labels.
        :param y: Labels.
        """
        hist = np.bincount(y)  # Compte les occurrences des classes.
        ps = hist / len(y)  # Probabilités de chaque classe.
        return -np.sum([p * np.log(p) for p in ps if p > 0])  # Entropie.

    def _most_common_label(self, y):
        """
        Trouve la classe majoritaire dans les labels.
        :param y: Labels.
        """
        counter = Counter(y)
        value = counter.most_common(1)[0][0]  # Classe avec le plus grand nombre d'occurrences.
        return value

    def predict(self, X):
        """
        Prédit les classes pour un ensemble de données.
        :param X: Données d'entrée.
        """
        # Parcourt chaque échantillon et traverse l'arbre pour prédire.
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        """
        Traverse l'arbre récursivement pour effectuer une prédiction.
        :param x: Un échantillon.
        :param node: Nœud actuel.
        """
        # Si le nœud est une feuille, retourne la valeur.
        if node.is_leaf_node():
            return node.value

        # Vérifie si l'échantillon va à gauche ou à droite.
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)


# Model training

In [4]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score

In [5]:
X,y = datasets.make_blobs(
    n_samples = 200, n_features = 2, centers = 2, cluster_std = 1.05 
)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)

In [7]:
model = DecisionTree()

In [8]:
model.fit(x_train, y_train)

In [9]:
predictions = model.predict(x_test)

In [10]:
accuracy_score(predictions, y_test)

1.0